# Projeto 1 - Ciência dos Dados

Nome: Arthur Fuhrken 

Nome: Pedro Mahfuz

Nome: Rodrigo Furukawa

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [23]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import emoji
from math import *

In [24]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\osbor\Documents\2021.1\Cdados DP\Projeto 1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [25]:
filename = 'ruffles.xlsx'

In [26]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Valor
0,uma criancinha deu um ruffles pra ela,1
1,rt @favskyliex: 📖 sorteio 📖\n\npara concorrer ...,0
2,"@_adryelleac @americanascom aí sim, adry! imag...",2
3,@b_ruffles eu vo para de estuda também,0
4,assimmm completamente eu arrumando as parede d...,0


In [27]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Valor
0,@vulgo_ruffles amo vc 🥰🥰,0
1,rt @igoritonobrasil: 🏐💕 amor doce 💕🏐\n#haikyuu...,0
2,rt @bonecker_: incrível que eu taco o fodase p...,2
3,"[9:55 pm, 17/03/2021] alexia: eu comi só ruffl...",1
4,hj não foi um bom dia,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto é a famosa batata "Ruffles", consideramos como tweets relevantes os que falassem do produto de uma maneira pejorativa ou positiva. Consideramos como irrelevantes, os tweets que falavam do produto, mas não agregavam nada a ele. Por fim, consideramos muito irrelevantes os tweets que não faziam qualquer tipo de menção ao produto ou que continham conteúdo completamente desagregado ao valor dele.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [28]:
#Criando classes "cleanup", usadas para deixar as strings mais padronizadas.

def cleanup_pontuacao(tweets):
    p = '[!-.:?;"\n"()''"",_%$®|=“”—/]'
    padrao = re.compile(p)
    remove_ponto = re.sub(padrao,' ',tweets)
    espaco = ' '
    padrao = re.compile(espaco)
    tweets_espaco = re.sub(padrao, ' ', remove_ponto)
    return tweets_espaco
    
def cleanup_espacos(tweets):    
    espacos = ' '
    padrao = re.compile(espacos)
    remove_espaco = re.sub(padrao,' ', tweets)
    return remove_espaco

def cleanup_link(tweets):
    link = r'http[^\s]*'
    padrao = re.compile(link)
    remove_link = re.sub(padrao,'',tweets)
    return remove_link

trein_str=str()
for n in train['Treinamento']:
    trein_str+=' '+n



trein = trein_str.lower()
trein1 = cleanup_pontuacao(trein)
trein2 = cleanup_espacos(trein1)
trein_limpo = cleanup_link(trein2)

In [29]:
#Usando UNICODE emoji para separar os emojis

def seleciona_emoji(character):
        return character in emoji.UNICODE_EMOJI 
    
def character_tem_emoji(texto):
    return ''.join(' ' + char + ' ' if seleciona_emoji(char) else char for char in texto).strip()    

trein_f = character_tem_emoji(trein_limpo)

In [30]:
#Criando um pd.Series com todas as palavras e suas frequencias relativas

palavras = trein_f.split()

total = pd.Series(palavras)
total.value_counts(True)

ruffles       0.046613
de            0.036663
e             0.025896
que           0.018400
a             0.017582
                ...   
batata…       0.000136
espinhando    0.000136
deve          0.000136
6iwz338win    0.000136
matar         0.000136
Length: 2186, dtype: float64

In [31]:
train['Valor']=train['Valor'].astype('category')
train['Valor'].cat.categories = ["Muito Irrelevante","Irrelevante","Relevante"]

train['Valor'].value_counts(True)

Muito Irrelevante    0.396
Irrelevante          0.330
Relevante            0.274
Name: Valor, dtype: float64

In [32]:
filtro_0 = train['Valor'] == 'Muito Irrelevante'
muito_irrelevante = train.loc[filtro_0,:]

filtro_1 = train['Valor'] == 'Irrelevante'
irrelevante = train.loc[filtro_1,:]

filtro_2 = train['Valor'] == 'Relevante'
relevante = train.loc[filtro_2,:]

In [33]:
#Limpando relevantes da tabela Treinamento com as funções criadas no inicio

relevantes_clean=str()
for n in relevante['Treinamento']:
    relevantes_clean+=' '+n
    
relevantes = relevantes_clean.lower()
relevantes1 = cleanup_pontuacao(relevantes)
relevantes2 = cleanup_espacos(relevantes1)
relevantes_limpo = cleanup_link(relevantes2)
relevantes_emoji = character_tem_emoji(relevantes_limpo)

palavras1 = relevantes_emoji.split()
total1 = pd.Series(palavras1)

In [34]:
#Limpando irrelevantes da tabela Treinamento com as funções criadas no inicio

irrelevantes_clean=str()
for n in irrelevante['Treinamento']:
    irrelevantes_clean+=' '+n
    
irrelevantes = irrelevantes_clean.lower()
irrelevantes1 = cleanup_pontuacao(irrelevantes)
irrelevantes2 = cleanup_espacos(irrelevantes1)
irrelevantes_limpo = cleanup_link(irrelevantes2)
irrelevantes_emoji = character_tem_emoji(irrelevantes_limpo)

palavras2 = irrelevantes_emoji.split()
total2 = pd.Series(palavras2)

In [35]:
#Limpando m_irrelevantes da tabela Treinamento com as funções criadas no inicio

m_irrelevantes_clean=str()
for n in muito_irrelevante['Treinamento']:
    m_irrelevantes_clean+=' '+n
    
m_irrelevantes = m_irrelevantes_clean.lower()
m_irrelevantes1 = cleanup_pontuacao(m_irrelevantes)
m_irrelevantes2 = cleanup_espacos(m_irrelevantes1)
m_irrelevantes_limpo = cleanup_link(m_irrelevantes2)
m_irrelevantes_emoji = character_tem_emoji(m_irrelevantes_limpo)

palavras3 = m_irrelevantes_emoji.split()
total3 = pd.Series(palavras3)

In [36]:
#Criando lista com palavras limpas relevantes

relevantes = []

for i in relevante['Treinamento']:
    relevantes_clean = i.lower()
    relevantes1 = cleanup_pontuacao(relevantes_clean)
    relevantes2 = cleanup_espacos(relevantes1)
    relevantes_limpo = cleanup_link(relevantes2)
    relevantes_emoji = character_tem_emoji(relevantes_limpo)
    
    for t in relevantes_emoji.split():
        relevantes.append(t)

In [37]:
#Criando lista com palavras limpas irrelevantes

irrelevantes = []

for i in irrelevante['Treinamento']:
    irrelevantes_clean = i.lower()
    irrelevantes1 = cleanup_pontuacao(irrelevantes_clean)
    irrelevantes2 = cleanup_espacos(irrelevantes1)
    irrelevantes_limpo = cleanup_link(irrelevantes2)
    irrelevantes_emoji = character_tem_emoji(irrelevantes_limpo)
    
    for t in irrelevantes_emoji.split():
        irrelevantes.append(t)    

In [38]:
#Criando lista com palavras limpas muito irrelevantes

m_irrelevantes = []

for i in muito_irrelevante['Treinamento']:
    m_irrelevantes_clean = i.lower()
    m_irrelevantes1 = cleanup_pontuacao(m_irrelevantes_clean)
    m_irrelevantes2 = cleanup_espacos(m_irrelevantes1)
    m_irrelevantes_limpo = cleanup_link(m_irrelevantes2)
    m_irrelevantes_emoji = character_tem_emoji(m_irrelevantes_limpo)
    
    for t in m_irrelevantes_emoji.split():
        m_irrelevantes.append(t)

In [39]:
#Criando lista com todas as palavras limpas

todas_palavras = []

for i in train['Treinamento']:
    todas_clean = i.lower()
    todas1 = cleanup_pontuacao(todas_clean)
    todas2 = cleanup_espacos(todas1)
    todas_limpo = cleanup_link(todas2)
    todas_emoji = character_tem_emoji(todas_limpo)
    
    for t in todas_emoji.split():
        todas_palavras.append(t)

In [40]:
# Somando palavras de cada categoria

rel = total1.value_counts()
total_rel = total1.value_counts().sum()

irrel = total2.value_counts()
total_irrel = total2.value_counts().sum()

m_irrel = total3.value_counts()
total_m_irrel = total3.value_counts().sum()

total_palavras = total.value_counts()
total_palavras_soma = total.value_counts().sum()

In [41]:
# Smoothing para cada categoria

A = len(total_palavras)
a = 1

smooth_0_rel = rel + a
smooth_1_rel = total_rel + A

smooth_0_irrel = irrel + a
smooth_1_irrel = total_irrel + A

smooth_0_m_irrel = m_irrel + a
smooth_1_m_irrel = total_m_irrel + A

len(total_palavras)

2186

In [42]:
# P(palavra|"Categoria")

P_pal_rel = smooth_0_rel/smooth_1_rel
P_pal_irrel = smooth_0_irrel/smooth_1_irrel
P_pal_m_irrel = smooth_0_m_irrel/smooth_1_m_irrel

In [43]:
p_relevantes = []
p_irrelevantes = []
p_m_irrelevantes = []

for i in relevantes:
    p_t = 0
    if i in P_pal_rel:
        p_t+=log(P_pal_rel[i])
    else:
        p_t+=log(a/total_rel+A)
    p_relevantes.append(p_t)


for i in irrelevantes:
    p_t = 0
    if i in P_pal_irrel:
        p_t+=log(P_pal_irrel[i])
    else:
        p_t+=log(a/total_irrel+A)
    p_irrelevantes.append(p_t)
    

for i in m_irrelevantes:
    p_t = 0
    if i in P_pal_m_irrel:
        p_t+=log(P_pal_m_irrel[i])
    else:
        p_t+=log(a/total_m_irrel+A)
    p_m_irrelevantes.append(p_t)

In [48]:
# Criando dicionários com keys = palavra, e values = probabilidade, de cada categoria

d_relevante = {}
for i in range(len(relevantes)):
    d_relevante[relevantes[i]] = p_relevantes[i]


d_irrelevante = {}
for i in range(len(irrelevantes)):
    d_irrelevante[irrelevantes[i]] = p_irrelevantes[i]


d_m_irrelevante = {}
for i in range(len(m_irrelevantes)):
    d_m_irrelevante[m_irrelevantes[i]] = p_m_irrelevantes[i]
    

# Criando código que identifica se uma palavra de ser considerada Relevante, Irrelevante ou Muito irrelevante     
    
d_final = {}

for k_r,v_r in d_relevante.items():
    for k_i,v_i in d_irrelevante.items():
        if k_r == k_i:
            for k_m,v_m in d_m_irrelevante.items():
                if k_r == k_m:
                    if (v_r > v_i) and (v_r > v_m):
                        d_final[k_r] = "Relevante"
                    elif (v_i > v_r) and (v_i > v_m):
                        d_final[k_i] = "Irrelevante"
                    elif (v_m > v_r) and (v_m > v_i):
                        d_final[k_m] = "Muito irrelevante"

            if (k_r != k_i) and (k_r != k_m) and k_r not in d_final:
                d_final[k_r] = 'Relevante'

            elif (k_r != k_i) and (k_r != k_m) and k_i not in d_final:
                d_final[k_i] = 'Irrelevante'

            elif (k_r != k_i) and (k_r != k_m) and k_m not in d_final:
                d_final[k_m] = 'Muito irrelevante'
d_final

{'@': 'Muito irrelevante',
 '@americanascom': 'Irrelevante',
 'aí': 'Muito irrelevante',
 'sim': 'Relevante',
 'comprar': 'Irrelevante',
 'uma': 'Relevante',
 'ruffles': 'Irrelevante',
 'tira': 'Irrelevante',
 'onda': 'Irrelevante',
 'por': 'Relevante',
 'apenas': 'Muito irrelevante',
 '99': 'Irrelevante',
 'centavos': 'Relevante',
 'isso': 'Muito irrelevante',
 'é': 'Relevante',
 'coisa': 'Relevante',
 'de': 'Relevante',
 'outro': 'Relevante',
 'mundo': 'Muito irrelevante',
 '🌀': 'Muito irrelevante',
 'o': 'Muito irrelevante',
 'eu': 'Muito irrelevante',
 'comi': 'Irrelevante',
 'só': 'Relevante',
 'um': 'Irrelevante',
 'pacote': 'Relevante',
 'batata': 'Relevante',
 'e': 'Irrelevante',
 'minha': 'Irrelevante',
 'foi': 'Irrelevante',
 'pra': 'Muito irrelevante',
 'quero': 'Relevante',
 'me': 'Relevante',
 'coca': 'Relevante',
 'bem': 'Muito irrelevante',
 'mim': 'Relevante',
 'em': 'Relevante',
 'que': 'Relevante',
 'salgadinho': 'Relevante',
 'da': 'Irrelevante',
 'ou': 'Irrelevante'

In [49]:
d_total ={}

for i in todas_palavras:
    for k_f,v_f in d_final.items():
        if k_f == i:
            d_total[i] = v_f
            
lista_final = []

for i in d_total.values():
    lista_final.append(i)
    
Naive_Bayes= pd.Series(lista_final)
train['Naive Bayes'] = Naive_Bayes 
pd.crosstab(train['Valor'], train['Naive Bayes'], normalize = 'index')


Naive Bayes,Irrelevante,Muito irrelevante,Relevante
Valor,,,
Muito Irrelevante,0.323529,0.352941,0.323529
Irrelevante,0.305556,0.333333,0.361111
Relevante,0.344828,0.293103,0.362069


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**